In [1]:
import os
os.environ['LOGS_DIRECTORY'] = '../eval_logs'

import sys
sys.path.insert(0, '..')

In [2]:
import json
import pandas as pd

from tqdm.auto import tqdm
from pydantic import BaseModel
from pydantic_ai import Agent

from logs import LOG_DIR

In [3]:
LOG_DIR.absolute()

WindowsPath('C:/Users/alexe/git/aihero/code/eval/../eval_logs')

In [6]:
async def evaluate_log_record(eval_agent, log_record):
    messages = log_record['messages']
    
    instructions = log_record['system_prompt']
    question = messages[0]['parts'][0]['content']
    answer = messages[-1]['parts'][0]['content']
    
    log_simplified = simplify_log_messages(messages)
    log = json.dumps(log_simplified)

    user_prompt = user_prompt_format.format(
        instructions=instructions,
        question=question,
        answer=answer,
        log=log
    )

    result = await eval_agent.run(user_prompt, output_type=EvaluationChecklist)
    return result.output 

In [8]:
len(eval_set)

10

In [12]:
eval_results = []

for log_record in tqdm(eval_set):
    eval_result = await evaluate_log_record(eval_agent, log_record)
    eval_results.append((log_record, eval_result))

  0%|          | 0/10 [00:00<?, ?it/s]

In [13]:
rows = []

for log_record, eval_result in eval_results:
    messages = log_record['messages']

    row = {
        'file': log_record['log_file'].name,
        'question': messages[0]['parts'][0]['content'],
        'answer': messages[-1]['parts'][0]['content'],
    }

    checks = {c.check_name: c.check_pass for c in eval_result.checklist}
    row.update(checks)

    rows.append(row)

In [15]:
df_evals = pd.DataFrame(rows)
df_evals.mean(numeric_only=True) * 100

instructions_follow    100.0
instructions_avoid     100.0
answer_relevant        100.0
answer_clear           100.0
answer_citations       100.0
completeness            70.0
tool_call_search       100.0
dtype: float64